In [1]:
#!/usr/bin/env python

'''
DESCRIPTION
-----------
    To export pathway order
    
RETURN
------
    {}. :  file
        
        
EXPORTED FILE(s) LOCATION
-------------------------
    .//{}/{EXPERIMENT}/{}.
'''

# importing default libraries
# import os, argparse, sys
# sys.path.append('./')
import os, sys
ROOT_DIR = os.path.dirname(os.path.dirname(os.path.abspath('__file__')))
os.chdir(ROOT_DIR)
sys.path.append(ROOT_DIR)

In [2]:
# importing scripts in scripts folder
from scripts import config as src
# importing default libraries
import numpy as np
import pandas as pd
import datetime as dt

import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import OneHotEncoder

rand_state = 91

time_start = dt.datetime.now().time().strftime('%H:%M:%S') # = time.time() dt.datetime.now().strftime('%Y%m%d_%I%M%S%p')

**** scripts/config.py IMPORTED!!!
**** PROJECT FOLDER ,  /home/pgundogdu/projects/signalization_prior_knowledge_based_nn


In [3]:
# # giving the model path
# model_path = '../data/output/result_NN/models/signaling_log1p/design_a1_signaling.h5'

dataset              = 'reference_log1p.pck'
bio_knowledge        = 'pbk_circuit_hsa_sig.txt'
experiment           = 'exper_melanoma'
analysis             = 'activation'

# defining biological knowledge layer based on the architecture of required model
bio_layer_number     = 1

# the output location
loc_output = os.path.join(src.DIR_REPORTS, analysis, experiment)
src.define_folder(loc_=loc_output)

FOLDER information,  /home/pgundogdu/projects/signalization_prior_knowledge_based_nn/reports/activation/exper_melanoma/


'/home/pgundogdu/projects/signalization_prior_knowledge_based_nn/reports/activation/exper_melanoma/'

In [4]:
df = pd.read_pickle(os.path.join(src.DIR_DATA_PROCESSED, experiment, dataset))

df_bio = pd.DataFrame(pd.read_csv(os.path.join(src.DIR_DATA_PROCESSED, bio_knowledge), index_col=0)).sort_index()

In [5]:
df_bio_with_df = pd.merge(left=pd.DataFrame(df.columns[:-1]).set_index(0)
                                 , right=df_bio
                                 , left_index=True
                                 , right_index=True
                                 , how='left').fillna(0.0)
df_bio_with_df

,P-hsa03320-62,P-hsa03320-45,P-hsa03320-43,P-hsa03320-23,P-hsa03320-32,P-hsa03320-8,P-hsa03320-7,P-hsa03320-9,P-hsa03320-39,P-hsa03320-38,...,P-hsa04976-38,P-hsa04976-37,P-hsa04976-36,P-hsa04976-34,P-hsa04976-41,P-hsa04976-16,P-hsa05100-55,P-hsa05100-33,P-hsa05100-15,P-hsa05100-24
0,,,,,,,,,,,,,,,,,,,,,
a1bg,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
a1cf,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
a2m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
a2ml1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
a4galt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zyg11a,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zyg11b,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zyx,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
print('Dataset cell type, ', df.groupby('cell_type').size())
print('Dataset shape                     , ', df.shape)
print('Biological knowledge shape        , ', df_bio.shape)
print('Biological knowledge with df shape, ', df_bio_with_df.shape)

print('\nDataset gene order first 10              ,', list(df.columns[:10]))
print('Biological knowledge gene order first 10, ', list(df_bio_with_df.index[:10].values))

Dataset cell type,  cell_type
B.cell         573
Macrophage     294
NK              64
T.CD4          599
T.CD8         1231
dtype: int64
Dataset shape                     ,  (2761, 17995)
Biological knowledge shape        ,  (2987, 1210)
Biological knowledge with df shape,  (17994, 1210)

Dataset gene order first 10              , ['a1bg', 'a1cf', 'a2m', 'a2ml1', 'a4galt', 'a4gnt', 'aaas', 'aacs', 'aadac', 'aadacl2']
Biological knowledge gene order first 10,  ['a1bg', 'a1cf', 'a2m', 'a2ml1', 'a4galt', 'a4gnt', 'aaas', 'aacs', 'aadac', 'aadacl2']


In [7]:
ohe = OneHotEncoder()
X = df.iloc[:, :-1].values
y = df.iloc[:, -1:].values
y_ohe = ohe.fit_transform(y).toarray()
# groups = y.reshape(1,-1)[0]

print(X.shape)
print(y.shape)
# print(groups.shape)

(2761, 17994)
(2761, 1)


In [8]:
split='train_test_split_tuningFalse_filtergeneFalse'
model_name ='circuits_signaling_2_1_layer'
model, model_pbk = src.loading_model(f'./models/{experiment}/{split}/design_{model_name}_reference_log1p_Adam_relu_0.h5', -1)
model_pbk.summary()

Loaded model!! ./models/exper_melanoma/train_test_split_tuningFalse_filtergeneFalse/design_circuits_signaling_2_1_layer_reference_log1p_Adam_relu_0.h5
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer1_input (InputLayer)    [(None, 17994)]           0         
_________________________________________________________________
layer1 (Dense)               (None, 1210)              21773950  
Total params: 21,773,950
Trainable params: 21,773,950
Non-trainable params: 0
_________________________________________________________________


In [9]:
y_pred = model_pbk.predict(X)
# creating dataframe which stores both prediction label of model and 'encode' information of given layer
df_activation_pred = pd.concat([pd.DataFrame(y_pred, columns=df_bio_with_df.columns)
                                , pd.DataFrame([ohe.categories_[0][i] for i in np.argmax(model.predict(X), axis=-1)],  columns=['cell_type']) ]
                              , axis=1)
df_activation_pred

,P-hsa03320-62,P-hsa03320-45,P-hsa03320-43,P-hsa03320-23,P-hsa03320-32,P-hsa03320-8,P-hsa03320-7,P-hsa03320-9,P-hsa03320-39,P-hsa03320-38,...,P-hsa04976-37,P-hsa04976-36,P-hsa04976-34,P-hsa04976-41,P-hsa04976-16,P-hsa05100-55,P-hsa05100-33,P-hsa05100-15,P-hsa05100-24,cell_type
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,B.cell
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.855719,B.cell
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,B.cell
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,B.cell
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.022858,B.cell
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2756,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,T.CD8
2757,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,T.CD8
2758,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,T.CD8
2759,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,T.CD8


## REMOVING NON-ACTIVE NODES

In [10]:
# finding non-active pathways (according to std values)
pathway_std_less_than_one = df_activation_pred.groupby('cell_type').std().sum()[df_activation_pred.groupby('cell_type').std().sum()<1].index
# printing information about non-active nodes
print('The number of non-active nodes is ', len(pathway_std_less_than_one))
print('The non-active nodes are; ')
print(pathway_std_less_than_one)

The number of non-active nodes is  1025
The non-active nodes are; 
Index(['P-hsa03320-62', 'P-hsa03320-45', 'P-hsa03320-43', 'P-hsa03320-23',
       'P-hsa03320-32', 'P-hsa03320-8', 'P-hsa03320-7', 'P-hsa03320-9',
       'P-hsa03320-39', 'P-hsa03320-38',
       ...
       'P-hsa04972-21', 'P-hsa04973-5', 'P-hsa04976-95 96', 'P-hsa04976-59',
       'P-hsa04976-36', 'P-hsa04976-34', 'P-hsa04976-41', 'P-hsa04976-16',
       'P-hsa05100-55', 'P-hsa05100-33'],
      dtype='object', length=1025)


## EXPORTING ACTIVE NODES' ACTIVATION SCORE

In [11]:
# export activation scores of each pathways for each sample
df_active = df_activation_pred.iloc[:, ~df_activation_pred.columns.isin(pathway_std_less_than_one)]
df_active.to_csv(os.path.join(loc_output, 'activation_score.csv'), index=False)
df_active

,P-hsa03320-40,P-hsa03320-10,P-hsa03320-31,P-hsa03320-59,P-hsa03320-56,P-hsa04010-118,P-hsa04010-131,P-hsa04010-64,P-hsa04010-61,P-hsa04010-15,...,P-hsa04971-56,P-hsa04971-39,P-hsa04972-42,P-hsa04972-39,P-hsa04973-36,P-hsa04976-38,P-hsa04976-37,P-hsa05100-15,P-hsa05100-24,cell_type
0,139.933655,12.280187,0.0,0.0,23.869902,0.0,0.0,16.818470,32.005371,0.0,...,2.194101,0.0,99.052147,0.0,0.000000,5.062793,0.0,0.0,0.000000,B.cell
1,74.825455,8.869648,0.0,0.0,21.001390,0.0,0.0,0.000000,0.000000,0.0,...,0.000000,0.0,55.504799,0.0,0.000000,0.000000,0.0,0.0,1.855719,B.cell
2,61.868420,5.778725,0.0,0.0,14.066553,0.0,0.0,1.234697,0.000000,0.0,...,0.000000,0.0,46.790356,0.0,0.000000,0.000000,0.0,0.0,0.000000,B.cell
3,177.857849,0.000000,0.0,0.0,55.661419,0.0,0.0,0.000000,52.673912,0.0,...,14.123025,0.0,121.452187,0.0,0.000000,0.000000,0.0,0.0,0.000000,B.cell
4,88.385643,13.896157,0.0,0.0,25.545576,0.0,0.0,0.000000,0.000000,0.0,...,0.000000,0.0,71.726349,0.0,0.000000,0.000000,0.0,0.0,7.022858,B.cell
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2756,58.679909,0.000000,0.0,0.0,0.000000,0.0,0.0,27.442432,16.118793,0.0,...,20.976530,0.0,45.019993,0.0,0.000000,1.201300,0.0,0.0,0.000000,T.CD8
2757,23.129784,0.111213,0.0,0.0,0.000000,0.0,0.0,13.126150,8.986176,0.0,...,12.579815,0.0,14.673282,0.0,5.605000,3.469063,0.0,0.0,0.000000,T.CD8
2758,29.806541,0.000000,0.0,0.0,0.000000,0.0,0.0,20.502556,7.304921,0.0,...,17.415239,0.0,23.106655,0.0,1.261880,0.597604,0.0,0.0,0.000000,T.CD8
2759,58.934578,0.000000,0.0,0.0,0.000000,0.0,0.0,29.246082,15.967497,0.0,...,21.755663,0.0,47.983208,0.0,0.000000,0.000000,0.0,0.0,0.000000,T.CD8


## EXPORTING CELL TYPE - PATHWAY RELATION and CELL TYPE - FREQUENT GENE INFORMATION

In [12]:
no_of_nodes_in_pathways=6
cell_path_info = [ohe.categories_[0][x] for x in np.argmax(model.layers[2].get_weights()[0], axis=-1)]
df_cell_path_info = pd.DataFrame()
df_gene_freq = pd.DataFrame()
for i_cell in df['cell_type'].unique():
    print(i_cell)
    cell_path_no = [i for i, x in enumerate(cell_path_info) if x == i_cell]
    # realted pathways with cell types
    df_temp = pd.DataFrame(df_bio_with_df[df_bio_with_df.columns[cell_path_no]].sum(axis=0), columns=['total_gene']).reset_index()
    df_temp['cell_type'] = i_cell
    df_cell_path_info = df_cell_path_info.append(df_temp)
    # the most frequent gene
    gene_freq = df_bio_with_df[df_bio_with_df.columns[cell_path_no]].sum(axis=1)[df_bio_with_df[df_bio_with_df.columns[cell_path_no]].sum(axis=1)>no_of_nodes_in_pathways].index
    df_temp = pd.DataFrame(gene_freq)
    df_temp['cell_type'] = i_cell
    df_gene_freq = df_gene_freq.append(df_temp)

df_gene_freq = df_gene_freq.rename(columns={0:'symbol'})
    
df_cell_path_info.to_csv(os.path.join(loc_output,'circuits_cell_type_relation.csv'), index=False)
df_gene_freq.to_csv(os.path.join(loc_output,'gene_cell_type_relation.csv'), index=False)

B.cell
Macrophage
NK
T.CD4
T.CD8


## EXPORTING MOST FREQUENT GENE LIST FOR OVERALL DATASET

In [13]:
# frequent more than {len(cell type)-1} cell type
df_freq_for_cell_type = df_gene_freq['symbol'].value_counts()[df_gene_freq['symbol'].value_counts()>=len(df['cell_type'].unique())-1]
# pd.DataFrame(df_freq_for_cell_type.index, columns=['gene']).to_csv(os.path.join(loc_output,'gene_most_freq_fig3g.csv'), index=False)
pd.DataFrame(df_gene_freq['symbol'].value_counts().index, columns=['gene']).to_csv(os.path.join(loc_output,'gene_most_freq.csv'), index=False)

In [14]:
df_gene_freq['symbol'].value_counts().index

Index(['pdgfa', 'pik3ca', 'traf2', 'csnk1e', 'fgfr2', 'fgfr3', 'plcb1', 'nras',
       'tek', 'epha2',
       ...
       'ptprf', 'fzd9', 'rasa1', 'phlpp2', 'prkci', 'fzd5', 'frmd1', 'map3k5',
       'stat5a', 'dusp16'],
      dtype='object', length=673)

## TOP 10 ACTIVE PATHWAYS for EACH CELL TYPES

In [15]:
top_active_pathway = 15
df_freq_pathway = pd.DataFrame()
for i in range(len(df_active)):
#     cell_type_info = df_active.iloc[i, -1]
    pathway_active = pd.DataFrame(pd.DataFrame(df_active.iloc[i, :-1][df_active.iloc[i, :-1]>0]).sort_values(by=i, ascending=False)[:top_active_pathway].index
                                  , columns=['pathways'])
    pathway_active['cell_type'] = df_active.iloc[i, -1]
    df_freq_pathway = df_freq_pathway.append(pathway_active)

In [16]:
df_freq_pathway_count = pd.DataFrame(df_freq_pathway.groupby(['pathways', 'cell_type']).size(), columns=['count']).reset_index()
# df_freq_pathway_count
df_freq_pathway_pivot = df_freq_pathway_count.pivot(index='pathways',columns='cell_type', values='count').fillna(0).reset_index()


In [17]:
top_n = 10
df_freq_pathway_top_n = pd.DataFrame()
for i_ct in df_freq_pathway['cell_type'].unique():
    df_temp = df_freq_pathway_pivot[['pathways',i_ct]].sort_values(by=i_ct,ascending=False)[:top_n][['pathways']]
    df_temp['cell_type']= i_ct
    df_freq_pathway_top_n = df_freq_pathway_top_n.append(df_temp)
    
df_freq_pathway_top_n.to_csv(os.path.join(loc_output,f'freq_circuits_top{top_n}.csv'), index=False)

In [18]:
len(df_freq_pathway_top_n)

50

## PLOT DISTRIBUTION of PATHWAYS ACTIVITIES

## EFFECTOR GENE ANALYSIS

In [10]:
df_effector_gene = pd.read_csv('./data/processed/circuits_effector.txt').set_index('circuits')
df_effector_final = df_effector_gene[df_effector_gene.index.isin(df_activation_pred.columns)]#.reset_index(drop=True)
df_effector_final['effector_gene_name'] = df_effector_final['effector_gene_name'].str.lower()
df_effector_final['effector_gene_name'].to_csv('effector_gene_name.txt')
df_effector_final

,effector_gene_name,effector_entrez
circuits,,
P-hsa03320-37,hmgcs2,3158
P-hsa03320-61,apoa1,335
P-hsa03320-46,apoa2,336
P-hsa03320-57,apoc3,345
P-hsa03320-64,apoa5,116519
...,...,...
P-hsa04976-95 96,slc51a slc51b,200931; /; 123264
P-hsa05100-24,actb,60; 71
P-hsa05100-15,ctnna1,1495; 1496; 29119


In [11]:
df_circuit_weight = pd.DataFrame(model.layers[1].get_weights()[0]
                                 , columns=df_effector_final.index
                                 , index=df_bio_with_df.index
                                )
# df_circuit_weight.to_csv('weight_scores_of_input_layer.txt')
df_circuit_weight

circuits,P-hsa03320-37,P-hsa03320-61,P-hsa03320-46,P-hsa03320-57,P-hsa03320-64,P-hsa03320-47,P-hsa03320-65,P-hsa03320-55,P-hsa03320-56,P-hsa03320-33,...,P-hsa04976-34,P-hsa04976-36,P-hsa04976-37,P-hsa04976-38,P-hsa04976-59,P-hsa04976-95 96,P-hsa05100-24,P-hsa05100-15,P-hsa05100-33,P-hsa05100-55
0,,,,,,,,,,,,,,,,,,,,,
a1bg,-0.000000,-0.000000,0.000000,0.000000,-0.000000,-0.000000e+00,0.000000,-0.000000,5.218737e-03,0.002827,...,0.010742,0.005998,0.000000,-0.000000e+00,0.000000e+00,-0.005228,-0.006005,0.000000e+00,-0.000117,0.006505
a1cf,0.000000,-0.006000,0.000000,0.005993,-0.000000,5.988850e-03,-0.005258,0.000000,-5.141885e-03,-0.004331,...,0.009668,-0.013268,-0.000000,-5.996234e-03,0.000000e+00,0.003553,-0.005998,-5.073317e-03,-0.006516,0.003350
a2m,-0.000000,-0.008338,-0.000000,0.006003,-0.000000,-5.551368e-03,-0.005344,-0.000000,5.224218e-03,-0.001427,...,-0.010935,0.015158,0.000000,-6.869164e-03,-5.552395e-03,-0.009097,-0.008475,-5.180266e-03,0.020502,0.004636
a2ml1,0.000000,-0.004081,0.005535,0.005974,0.000000,-1.301493e-03,-0.000000,-0.006000,5.197317e-03,-0.001813,...,-0.003395,-0.008512,-0.005996,-5.951019e-03,-5.556155e-03,0.001713,-0.004524,-5.153919e-03,0.008430,0.006301
a4galt,0.000000,0.000000,0.000000,-0.000000,-0.000000,-0.000000e+00,-0.000000,-0.000000,0.000000e+00,0.000000,...,-0.018734,-0.020682,-0.000000,-0.000000e+00,0.000000e+00,0.000000,-0.000000,0.000000e+00,-0.000011,0.013375
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zyg11a,-0.005334,-0.004274,-0.005557,-0.006000,-0.005346,-3.656110e-03,-0.005339,-0.000000,-5.933449e-03,-0.002549,...,0.002477,-0.000528,-0.006001,-5.704518e-03,-5.557872e-03,-0.002490,-0.003803,-5.177253e-03,0.001923,-0.001865
zyg11b,-0.005324,-0.003566,-0.005557,-0.005999,-0.005346,-3.527719e-03,-0.005343,-0.006001,-6.528827e-03,-0.002962,...,0.002668,0.000846,-0.006001,-3.807036e-03,-5.557028e-03,-0.001213,-0.003720,-5.174365e-03,-0.002458,-0.001030
zyx,-0.000000,-0.008233,-0.005557,0.006004,-0.005346,-3.193313e-03,-0.005345,-0.000000,-5.885858e-03,0.005330,...,0.007732,-0.003534,-0.000000,-4.466941e-03,-0.000000e+00,-0.006824,-0.008477,0.000000e+00,0.005308,-0.003590


In [12]:
top_n = 100
df_active_gene_top_n = pd.melt(df_circuit_weight.apply(lambda s: s.nlargest(top_n).index.tolist(), axis=0)).set_index('circuits')
df_active_gene_top_n

,value
circuits,
P-hsa03320-37,rxra
P-hsa03320-37,rxrb
P-hsa03320-37,rxrg
P-hsa03320-37,a1bg
P-hsa03320-37,a1cf
...,...
P-hsa05100-55,stard5
P-hsa05100-55,pfkfb2
P-hsa05100-55,btk


In [13]:
df_effector_vs_active = pd.merge(df_active_gene_top_n, df_effector_final['effector_gene_name']
                                 , how='inner', left_index=True, right_index=True)
df_effector_vs_active

,value,effector_gene_name
circuits,,
P-hsa03320-10,rxra,plin1
P-hsa03320-10,rxrb,plin1
P-hsa03320-10,dbi,plin1
P-hsa03320-10,gsto1,plin1
P-hsa03320-10,cd4,plin1
...,...,...
P-hsa05100-55,stard5,actb*
P-hsa05100-55,pfkfb2,actb*
P-hsa05100-55,btk,actb*


In [15]:
df_effector_vs_active['common']=df_effector_vs_active.apply(lambda x: x.value in x.effector_gene_name, axis=1)
df_effector_vs_active

,value,effector_gene_name,common
circuits,,,
P-hsa03320-10,rxra,plin1,False
P-hsa03320-10,rxrb,plin1,False
P-hsa03320-10,dbi,plin1,False
P-hsa03320-10,gsto1,plin1,False
P-hsa03320-10,cd4,plin1,False
...,...,...,...
P-hsa05100-55,stard5,actb*,False
P-hsa05100-55,pfkfb2,actb*,False
P-hsa05100-55,btk,actb*,False


In [16]:
df_effector_vs_active[df_effector_vs_active['common']==True]

,value,effector_gene_name,common
circuits,,,
P-hsa03320-21,acaa1,acaa1,True
P-hsa04014-97,ak4,pak4,True
P-hsa04024-91,gria1,gria1,True
P-hsa04064-114,bcl2a1,bcl2a1*,True
P-hsa04064-51,aip,tnfaip3,True
P-hsa04068-66,bnip3,bnip3,True
P-hsa04110-48,rb1,rb1*,True
P-hsa04114-80,cdc25c,cdc25c*,True
P-hsa04115-25,cd82,cd82,True
